In [61]:
config = {
    'input_path': "first_submissions/UvA.ILPS_bundle2_FR_2.tsv.mentions_only",
    'output_path': "first_submissions/UvA.ILPS_bundle2_FR_2.tsv.nel_only"
}

In [62]:
FILE_PATH = config['input_path']

OUTPUT_PATH = config['output_path']

Step 0: reading

In [63]:
from utils.data_processing import read_data_to_dfs, write_results, add_beginnings

In [64]:
import pickle

def get_dfs(path):
    try:
        run_name = path.split('/')[-1]
        folder = path.split('/')[0]
        pickle_path = folder + '/pickles/' + run_name + '.p'
#         print(pickle_path)
        dfs = pickle.load(open(pickle_path, 'rb'))
    except Exception:
        print('pickled file not found')
        dfs = read_data_to_dfs(path)
        
    return dfs

In [65]:
dfs_with_predictions = get_dfs(FILE_PATH)

# import pickle
# dfs_with_predictions = pickle.load(open(FILE_PATH+'.p', 'rb'))

In [66]:
dfs_with_predictions[1145]

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
0,quarantaines,O,_,_,_,_,_,_,_,_
1,des,O,_,_,_,_,_,_,_,_
2,ports,O,_,_,_,_,_,_,_,_
3,de,O,_,_,_,_,_,_,_,_
4,France,B-ent,_,_,_,_,_,_,_,_
5,et,O,_,_,_,_,_,_,_,_
6,d,O,_,_,_,_,_,_,_,NoSpaceAfter
7,',O,_,_,_,_,_,_,_,NoSpaceAfter
8,Italie,B-ent,_,_,_,_,_,_,_,_
9,si,O,_,_,_,_,_,_,_,_


In [67]:
extract_entity_mentions(dfs_with_predictions[1145])

[(4, 'France'), (8, 'Italie')]

Step 1: NEL

In [68]:
from elasticsearch import Elasticsearch

es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)



In [69]:
es.info()

{'name': 'ilps-cn001',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'vU7MO42lR2KzA6bPvSP_pA',
 'version': {'number': '7.7.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '81a1e9eda8e6183f5237786246f6dced26a10eaf',
  'build_date': '2020-05-12T02:01:37.602180Z',
  'build_snapshot': False,
  'lucene_version': '8.5.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [70]:
from utils.data_processing import extract_entity_mentions

In [71]:
from utils.elastic_getters import wikidata_search_, wikidata_search_precise

In [72]:
# Search for an entity
def get_wikidata_entries(es, entity):
    hits = wikidata_search_(es, entity)
    hits_exact = wikidata_search_precise(es, entity)
    
    if len(hits) + len(hits_exact) == 0:
        return "NIL"
    
    answer_length = min(5, len(hits_exact))
    results = [hit['_source']['label_exact'] for hit in hits_exact[:answer_length]]
    remaining_length = 5 - answer_length
    
    if remaining_length > 0:
        results.append('NIL')
        remaining_length -= 1
        
    remaining_hits = [hit for hit in hits if hit['_source']['label_exact'] not in results]
    

    if remaining_length > 0:
        remaining_hits_sorted = sorted(remaining_hits, key=lambda x: (-x['_score'], len(x['_source']['label_exact']), x['_source']['count']))
        cnt = 0
        while remaining_length > 0 and cnt < len(remaining_hits_sorted):
            results.append(remaining_hits_sorted[cnt]['_source']['label_exact'])
            cnt += 1
            remaining_length -= 1
            
    return '|'.join(results)

In [73]:
def insert_links(df, ner_column='NE-COARSE-LIT', nel_column='NEL-LIT'):
    res_df = df.copy()
    true_labels = df[nel_column].tolist()
    mentions = extract_entity_mentions(df, ner_column)
    
    for position, mention in mentions:
        labels = get_wikidata_entries(es, mention)
        cur_pos = position
        num_words = len(mention.split())
        while cur_pos - position < num_words:
            res_df[nel_column][cur_pos] = labels
            cur_pos += 1
    return res_df

In [75]:
df_test=insert_links(dfs_with_predictions[1145])
df_test

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
0,quarantaines,O,_,_,_,_,_,_,_,_
1,des,O,_,_,_,_,_,_,_,_
2,ports,O,_,_,_,_,_,_,_,_
3,de,O,_,_,_,_,_,_,_,_
4,France,B-ent,_,_,_,_,_,Q142|Q3080569|Q16275867|Q5478259|Q294485,_,_
5,et,O,_,_,_,_,_,_,_,_
6,d,O,_,_,_,_,_,_,_,NoSpaceAfter
7,',O,_,_,_,_,_,_,_,NoSpaceAfter
8,Italie,B-ent,_,_,_,_,_,Q3155865|NIL|Q86253379|Q49564862|Q1961141,_,_
9,si,O,_,_,_,_,_,_,_,_


In [76]:
dfs_with_links = [insert_links(df) for df in dfs_with_predictions]

No beginning found! Token: sieur
No beginning found! Token: 1797
No beginning found! Token: Landeron
No beginning found! Token: sieur
No beginning found! Token: Neuchatel
No beginning found! Token: Fleurier
No beginning found! Token: Jouas
No beginning found! Token: Lequin
No beginning found! Token: Neuchatel
No beginning found! Token: Stnuffer
No beginning found! Token: Mlle
No beginning found! Token: demoiselle
No beginning found! Token: Crêt
No beginning found! Token: Neuchatel
No beginning found! Token: charpentier
No beginning found! Token: sieur
No beginning found! Token: Neuchatel
No beginning found! Token: Chaumont
No beginning found! Token: chtz
No beginning found! Token: Ghatenay
No beginning found! Token: secrétaire
No beginning found! Token: l
No beginning found! Token: de
No beginning found! Token: Robert
No beginning found! Token: jft
No beginning found! Token: sieur
No beginning found! Token: Jaques
No beginning found! Token: Verrières
No beginning found! Token: Cote
No 

No beginning found! Token: Kellogg
No beginning found! Token: LONDRES
No beginning found! Token: Havas
No beginning found! Token: M
No beginning found! Token: Venizelos
No beginning found! Token: Szen
No beginning found! Token: BELGRADE
No beginning found! Token: Havas
No beginning found! Token: Novosti
No beginning found! Token: PARIS
No beginning found! Token: Havas
No beginning found! Token: Silésie
No beginning found! Token: «
No beginning found! Token: Zeitung
No beginning found! Token: Silésie
No beginning found! Token: Schweitnitz
No beginning found! Token: Schweitnitz
No beginning found! Token: Weistritz
No beginning found! Token: Riesengebirge
No beginning found! Token: STOCKHOLM
No beginning found! Token: Toujquet
No beginning found! Token: PARIS
No beginning found! Token: Havas
No beginning found! Token: _
No beginning found! Token: Mme
No beginning found! Token: Truay
No beginning found! Token: BELGRADE
No beginning found! Token: Avala
No beginning found! Token: aulrèvilles

No beginning found! Token: Lanterne
No beginning found! Token: Avis
No beginning found! Token: Walewski
No beginning found! Token: bachelier
No beginning found! Token: procureur
No beginning found! Token: catholicisme
No beginning found! Token: Moniteur
No beginning found! Token: Prusse
No beginning found! Token: Mayence
No beginning found! Token: Diable
No beginning found! Token: Quatre
No beginning found! Token: Lanterne
No beginning found! Token: La
No beginning found! Token: oonlre
No beginning found! Token: Prusse
No beginning found! Token: Prusse
No beginning found! Token: Prusse
No beginning found! Token: Nouvelliste
No beginning found! Token: Bohême
No beginning found! Token: de
No beginning found! Token: Gallicie
No beginning found! Token: landwehr
No beginning found! Token: de
No beginning found! Token: Bohême
No beginning found! Token: Havane
No beginning found! Token: d
No beginning found! Token: de
No beginning found! Token: Moudon
No beginning found! Token: Chàtillens
No 

No beginning found! Token: Hambour
No beginning found! Token: Mailau
No beginning found! Token: RUSSIE
No beginning found! Token: Bosdani
No beginning found! Token: Mauseï
No beginning found! Token: Vilna
No beginning found! Token: Soboline
No beginning found! Token: ANGLETERRE
No beginning found! Token: Devinez
No beginning found! Token: Eài
No beginning found! Token: JOUR
No beginning found! Token: Tchécoslovaquie
No beginning found! Token: Godesberg
No beginning found! Token: Godesberg
No beginning found! Token: de
No beginning found! Token: chancelier
No beginning found! Token: Tchécoslovaquie
No beginning found! Token: Fuhrer
No beginning found! Token: Tchécoslovaquie
No beginning found! Token: Sapin
No beginning found! Token: Sapin
No beginning found! Token: Motta
No beginning found! Token: Rodolphe
No beginning found! Token: Chaudet
No beginning found! Token: au
No beginning found! Token: Chaudet
No beginning found! Token: Jean
No beginning found! Token: IMPARTIAL
No beginning f

In [77]:
dfs_with_links[100]

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
0,moyens,O,_,_,_,_,_,_,_,_
1,d,O,_,_,_,_,_,_,_,NoSpaceAfter
2,',O,_,_,_,_,_,_,_,NoSpaceAfter
3,opposition,O,_,_,_,_,_,_,_,_
4,à,O,_,_,_,_,_,_,_,_
5,apporter,O,_,_,_,_,_,_,_,_
6,à,O,_,_,_,_,_,_,_,_
7,la,O,_,_,_,_,_,_,_,_
8,dite,O,_,_,_,_,_,_,_,_
9,demande,O,_,_,_,_,_,_,_,_


In [78]:
import pickle

run_name = OUTPUT_PATH.split('/')[-1]
folder = OUTPUT_PATH.split('/')[0]
pickle_path = folder + '/pickles/' + run_name + '.p'

pickle.dump(dfs_with_links, open(pickle_path, 'wb'))

In [79]:
write_results(dfs_with_links, OUTPUT_PATH)

In [16]:
'''
TEST_bundle2_FR_1 – no Europeana yet, 10 epochs
TEST_bundle2_FR_2 – with Europeana, 5 epochs
TEST_bundle2_FR_3 – no Europeana, 5 epochs - just to compare

TEST_bundle2_FR_4 – no Europeana, 8 epochs (it was the best for FR so far)
TEST_bundle2_FR_5 – with Europeana, 8 epochs - is it better? probably yes!

TEST_bundle2_DE_2 – with Europeana, 5 epochs
TEST_bundle2_DE_3 – no Europeana, 7 epochs
TEST_bundle4_DE_4 – with Europeana, 7 epochs


TEST_bundle2_EN_1 – BERT finetuned on conll English 

'''

'\nTEST_bundle2_FR_1 – no Europeana yet, 10 epochs\nTEST_bundle2_FR_2 – with Europeana, 5 epochs\nTEST_bundle2_FR_3 – no Europeana, 5 epochs - just to compare\n\nTEST_bundle2_FR_4 – no Europeana, 8 epochs (it was the best for FR so far)\nTEST_bundle2_FR_5 – with Europeana, 8 epochs - is it better? probably yes!\n\nTEST_bundle2_DE_2 – with Europeana, 5 epochs\nTEST_bundle2_DE_3 – no Europeana, 7 epochs\nTEST_bundle4_DE_4 – with Europeana, 7 epochs\n\n\nTEST_bundle2_EN_1 – BERT finetuned on conll English \n\n'